Model management with model registry in mlflow

In [10]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Handle class imbalance

In [13]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [1]:
import mlflow

In [10]:
# Initialize MLflow
mlflow.set_experiment("Anomaly_Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/12/16 17:20:02 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly_Detection' does not exist. Creating a new experiment.


2025/12/16 17:20:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/4/runs/0df0b54eae464264b53e14e88ce909e8
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/16 17:20:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/4/runs/970e3cd2e6ad49b5a2db2fe4ff3a7849
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/16 17:20:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/16 17:20:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/4/runs/867d2b9d166644f9ad620057386b329c
🧪 View experiment at: http://localhost:5000/#/experiments/4
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/4/runs/7172ce3e1e5d4b28b0a9047d6a1e8c96
🧪 View experiment at: http://localhost:5000/#/experiments/4


Register the Model

In [ ]:
model_name = 'XGB_Smote'
run_id="7172ce3e1e5d4b28b0a9047d6a1e8c96"
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

#ran the same code again, but with different run id, the version 2 got added

Successfully registered model 'XGB_Smote'.
2025/12/16 19:09:06 WARNING mlflow.tracking._model_registry.fluent: Run with id 7172ce3e1e5d4b28b0a9047d6a1e8c96 has no artifacts at artifact path 'model', registering model based on models:/m-72884a645f95417cb5fd425c8f3ecd6d instead
Created version '1' of model 'XGB_Smote'.


Load the Model

In [ ]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}" # or f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Transition the Model to Production

In [15]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB_Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1765893551622, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1765893551622, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='7172ce3e1e5d4b28b0a9047d6a1e8c96', run_link=None, source='models:/XGB_Smote/1', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [16]:
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])